In [1]:
import pandas as pd
import matplotlib

og_data = pd.read_csv(r'data/Warehouse_and_Retail_Sales.csv')

In [2]:
og_data.info()
og_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128355 entries, 0 to 128354
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   YEAR              128355 non-null  int64  
 1   MONTH             128355 non-null  int64  
 2   SUPPLIER          128331 non-null  object 
 3   ITEM CODE         128355 non-null  object 
 4   ITEM DESCRIPTION  128355 non-null  object 
 5   ITEM TYPE         128354 non-null  object 
 6   RETAIL SALES      128355 non-null  float64
 7   RETAIL TRANSFERS  128355 non-null  float64
 8   WAREHOUSE SALES   128355 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 8.8+ MB


,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0
